In [ ]:
import matplotlib.pyplot as plt
import xarray as xr

In [ ]:
ds = xr.open_dataset("https://data.source.coop/aldenks/gfs-dynamical/analysis/v0.1.0.zarr", engine="zarr", chunks={})
ds

In [ ]:
ds['temperature_2m'].sel(time='2024-05-01T00:00').plot(figsize=(12,8))

In [ ]:
tmp_april_mean = ds['temperature_surface'].sel(time=ds.time.dt.month == 4).mean(dim='time')

tmp_april_2024 = ds['temperature_surface'].sel(time='2024-04').mean(dim='time')

# April 2024 temperature anomaly
(tmp_april_2024 - tmp_april_mean).plot(figsize=(12, 8))

In [ ]:
(
    ds['temperature_surface'].sel(
        time='2024-04-01T00:00',
        latitude=slice(70, 20),
        longitude=slice(-150, -30)
    )
    .plot(figsize=(12, 8))
)

In [ ]:
(
    ds['precipitation_surface'].sel(
        latitude=slice(70, 20),
        longitude=slice(-150, -30)
    )
    .mean(dim='time')
    .plot(figsize=(12, 8))
)

In [ ]:
(
    (ds['precipitation_surface'].sel(
        time=slice('2024-01-01', '2024-05-01'),
    )
    .sum(dim='time') ** 0.5)
    .plot(figsize=(12, 8))
)

In [ ]:
wind_speed = (ds['wind_u_10m_above_ground']**2 + ds['wind_v_10m_above_ground']**2) ** 0.5
wind_speed.sel(time='2024-04-10T00').plot(figsize=(13, 8), cmap='YlGnBu_r')

In [ ]:
wind_speed = (ds['wind_u_10m_above_ground']**2 + ds['wind_v_10m_above_ground']**2) ** 0.5
ws = wind_speed.sel(time=slice('2023-01-01T00:00', '2024-01-01T00:00'))
ws.load()
ws

In [ ]:
from matplotlib.animation import FuncAnimation

w = ws.sel(time=slice('2023-01-01T00', '2023-02-01T00'))

def get_title(frame_da):
    return f"NOAA GFS Wind Speed {frame_da.time.dt.strftime('%Y-%m-%d').item()}"


fig, ax = plt.subplots(figsize=(1440/150 - 0.01, 721/150))

frame_0 = w.isel(time=0)
img = ax.imshow(frame_0.values, cmap='YlGnBu_r')
title = ax.set_title(get_title(frame_0), y=.94, color='white')

ax.axis("off")
fig.subplots_adjust(0, 0, 1, 1)

def update_animation(frame_da):
    img.set_data(frame_da.values)
    title.set_text(get_title(frame_da))

anim = FuncAnimation(fig=fig, func=update_animation, frames=w, interval=int(1000/15))
anim.save("gfs-wind-speed-2023-jan.webp", writer='imagemagick')

In [ ]:
p = ds['precipitation_surface'].sel(time=slice('2023-01-01T00:00', '2023-02-01T00:00'))
p.load()

In [ ]:
import cmocean
from matplotlib.animation import FuncAnimation

anim_da = p

def get_title(frame_da):
    return f"NOAA GFS Precipitation {frame_da.time.dt.strftime('%Y-%m-%d').item()}"


fig, ax = plt.subplots(figsize=(1440/150 - 0.01, 721/150))

frame_0 = anim_da.isel(time=0)
img = ax.imshow(frame_0.values, cmap=cmocean.cm.thermal, vmin=0, vmax=5e-4)
title = ax.set_title(get_title(frame_0), y=.94, color='white')

ax.axis("off")
fig.subplots_adjust(0, 0, 1, 1)

def update_animation(frame_da):
    img.set_data(frame_da.values)
    title.set_text(get_title(frame_da))

anim = FuncAnimation(fig=fig, func=update_animation, frames=anim_da, interval=int(1000/15))
anim.save("gfs-precipitation-2023-jan.webp", writer='imagemagick')

In [ ]:
t = ds['temperature_surface'].sel(time=slice('2023-01-01T00:00', '2023-02-01T00:00'))
t.load()

In [ ]:
import cmocean
from matplotlib.animation import FuncAnimation

anim_da = t.sel(latitude=slice(72, 22), longitude=slice(-23, 161))

def get_title(frame_da):
    return f"NOAA GFS Temperature {frame_da.time.dt.strftime('%Y-%m-%d').item()}"


fig, ax = plt.subplots(figsize=(len(anim_da.longitude)/70 - 0.01, len(anim_da.latitude)/70))

frame_0 = anim_da.isel(time=0)
img = ax.imshow(frame_0.values, cmap=cmocean.cm.balance, vmin=-50, vmax=50)
title = ax.set_title(get_title(frame_0), y=.9, color='black')

ax.axis("off")
fig.subplots_adjust(0, 0, 1, 1)

def update_animation(frame_da):
    img.set_data(frame_da.values)
    title.set_text(get_title(frame_da))

anim = FuncAnimation(fig=fig, func=update_animation, frames=anim_da, interval=int(1000/10))
anim.save("gfs-temperature-2023-jan.webp", writer='imagemagick')